In [1]:
using JSON
using HDF5
using Interact
using Plots
using StaticArrays
using TrafficWeavingPlanner
using DifferentialDynamicsModels
using LinearDynamicsModels

if isfile("config.json")
    config = JSON.parsefile("config.json")
else
    error("Please run setup.py in this directory before running any .ipynb's.")
end

Dict{String,Any} with 4 entries:
  "catkin_ws_dir"           => "/home/schmrlng/catkin_ws"
  "models_dir"              => "/data/trafficweaving/models"
  "feature_extraction_dict" => "slim"
  "data_dir"                => "/data/trafficweaving"

In [2]:
array_name = "trajectories_$(config["feature_extraction_dict"])"
base_filename = joinpath(config["data_dir"], array_name)
train_data = Dict(k => permutedims(v, ndims(v):-1:1) for (k,v) in h5open(read, base_filename * "_train.h5"))
eval_data = Dict(k => permutedims(v, ndims(v):-1:1) for (k,v) in h5open(read, base_filename * "_eval.h5"))

data_dict = eval_data    # as opposed to train_data, for testing
for (i, l) in enumerate(data_dict["traj_lengths"])               # hacky extrapolation
    data_dict["car2"][i,l+1:end,2] = data_dict["car2"][i,l,2]    # maintain y (derivatives are 0)
    data_dict["car2"][i,l+1:end,3] = data_dict["car2"][i,l,3]    # maintain ẋ (ẍ is 0)
    data_dict["car2"][i,l+1:end,1] = data_dict["car2"][i,l,1] +
                                     0.1*data_dict["car2"][i,l,3]*(1:size(data_dict["car2"],2)-l)
end
nothing

In [3]:
y_top = Float32(mean([y for y in train_data["car1"][:,1,2] if y > -4]))
y_bot = Float32(mean([y for y in train_data["car1"][:,1,2] if y < -4]))
xdd_choices = Float32[0, -3, 4, -6]
y_targets = [y_top, y_bot]
tibvp = SteeringBVP(TripleIntegratorDynamics(1, Float32), TimePlusQuadraticControl(SMatrix{1,1}(Float32(1e-3))))
nothing

  likely near In[3]:5
  likely near In[3]:5


In [4]:
include("traffic_weaving_control.jl")

dynamic_trial (generic function with 2 methods)

In [5]:
visualize_predictions(data_dict)

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -150 
 
 
 -100 
 
 
 -50 
 
 
 0 
 
 
 -8 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 position 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -100 
 
 
 -90 
 
 
 -80 
 
 
 -70 
 
 
 -60 
 
 
 -2.8 
 
 
 -2.6 
 
 
 -2.4 
 
 
 -2.2 
 
 
 -2.0 
 
 
 computation time = 1.798983087s 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 30 
 
 
 32 
 
 
 34 
 
 
 36 
 
 
 longitudinal velocity 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 30 
 
 
 32 
 
 
 34 
 
 
 36 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0.0 
 
 
 0.5 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 -5.0 
 
 
 -2.5 
 
 
 0.0 
 
 
 2.5 
 
 
 5.0 
 
 
 longitudinal acceleration 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 1.0 
 
 
 1.5 
 
 
 2.0 
 
 
 2.5 
 
 
 -6 
 
 
 -4 
 
 
 -2 
 
 
 0 
 
 
 2

In [6]:
idx = 222
car1_policy = static_trial(data_dict, idx, xdd_choices, y_targets, tibvp, 53-15)
t = plot([car1_policy[1,:,1], data_dict["car2"][idx,:,1]],
         [car1_policy[1,:,2], data_dict["car2"][idx,:,2]], marker=:x)
b = plot([car1_policy[1,:,5], data_dict["car2"][idx,:,5]])
plot(t, b, layout=@layout([a; b]), size=(500,800))

<?xml version="1.0" encoding="utf-8"?>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -125 
 
 
 -100 
 
 
 -75 
 
 
 -50 
 
 
 -25 
 
 
 0 
 
 
 25 
 
 
 -6 
 
 
 -5 
 
 
 -4 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
<polyline clip-path="url(#clip4102)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 54.5929,365.368 61.6858,365.511 68.8228,365.51 75.9598,365.511 83.0968,365.511 90.2338,365.511 97.3709,365.511 104.508,365.511 111.645,365.511 118.782,365.236 
 125.968,363.475 133.25,359.18 140.63,352.347 148.107,345.304 155.682,340.004 163.353,337.444 171.073,337.85 178.793,340.86 186.513,345.709 194.282,351.41 
 202.147,356.932 210.11,361.391 218.17,364.223 226.328,365.383 234.582,365.08 242.934,362.495 251.382,356.848 259.928,347.754 268.571,335.166 277.312,319.327 
 286.149,300.713 295.084,279.983 304.115,257.926 313.244,235.407 322.47,213.314 331.794,192.511 341.214,173.78 350.647,157.775 360.006,144.96 369.293,135.561 
 378.507,129.513 387.648,126.405 396.716,125.434 405.712,125.38 414.634,125.38 423.484,125.38 432.297,125.38 441.11,125.38 449.923,125.38 458.736,125.38 
 467.549,125.38 377.53,22.113 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
<polyline clip-path="url(#clip4102)" style="stroke:#e26f46; stroke-width:1; stroke-opacity:1; fill:none" points="
 42.0249,130.574 49.5112,133.002 56.9977,135.812 64.5082,138.841 72.0494,141.858 79.608,144.76 87.1877,147.747 94.8079,151.116 102.47,154.986 110.163,159.337 
 117.862,164.1 125.552,169.126 133.232,174.333 140.902,179.836 148.562,185.734 156.211,192.035 163.851,198.743 171.486,205.867 179.147,213.438 186.847,221.467 
 194.584,229.929 202.361,238.744 210.178,247.833 218.042,257.099 225.956,266.386 233.921,275.511 241.938,284.321 250.004,292.754 258.099,300.775 266.193,308.288 
 274.278,315.276 282.353,321.811 290.419,327.963 298.474,333.762 306.52,339.188 314.557,344.153 322.583,348.579 330.6,352.495 338.608,355.963 346.605,359.024 
 354.593,361.73 362.575,361.73 370.558,361.73 378.541,361.73 386.523,361.73 394.506,361.73 402.489,361.73 410.471,361.73 418.454,361.73 426.436,361.73 
 434.419,361.73 442.402,361.73 450.384,361.73 
 "/>
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 y1 
 
 
 
 
 
 
 
 y2 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 10 
 
 
 20 
 
 
 30 
 
 
 40 
 
 
 50 
 
 
 -3 
 
 
 -2 
 
 
 -1 
 
 
 0 
 
 
 1 
 
 
 2 
 
 
 3 
 
 
 4 
 
<polyline clip-path="url(#clip4103)" style="stroke:#009af9; stroke-width:1; stroke-opacity:1; fill:none" points="
 29.2591,430.693 37.9333,619.812 46.6074,619.812 55.2816,619.812 63.9557,619.812 72.6299,619.812 81.304,619.812 89.9782,619.812 98.6523,619.812 107.326,411.811 
 116.001,411.811 124.675,411.811 133.349,411.811 142.023,411.811 150.697,411.811 159.371,619.812 168.046,619.812 176.72,619.812 185.394,411.811 194.068,411.811 
 202.742,411.811 211.416,411.811 220.09,411.811 228.765,411.811 237.439,411.811 246.113,411.811 254.787,411.811 263.461,411.811 272.135,411.811 280.81,411.811 
 289.484,411.811 298.158,411.811 306.832,411.811 315.506,411.811 324.18,411.811 332.854,411.811 341.529,775.813 350.203,775.813 358.877,775.813 367.551,775.813 
 376.225,7